<a href="https://colab.research.google.com/github/aravind-sundaresan/huggingface-examples/blob/master/tweet_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! pip install transformers

In [9]:
from google.colab import drive
drive.mount("/drive")

import pandas as pd
from sklearn.model_selection import train_test_split
import tensorflow as tf
from transformers import DistilBertTokenizerFast
from transformers import TFDistilBertForSequenceClassification

Drive already mounted at /drive; to attempt to forcibly remount, call drive.mount("/drive", force_remount=True).


## Loading the input data
The input dataset consists of tweets pertaining to disasters.

In [3]:
tweet_df = pd.read_csv("data/tweets.csv")

tweet_df.head()

,id,keyword,location,text,target
0,0,ablaze,NaN,"Communal violence in Bhainsa, Telangana. ""Ston...",1
1,1,ablaze,NaN,Telangana: Section 144 has been imposed in Bha...,1
2,2,ablaze,New York City,Arsonist sets cars ablaze at dealership https:...,1
3,3,ablaze,"Morgantown, WV",Arsonist sets cars ablaze at dealership https:...,1
4,4,ablaze,NaN,"""Lord Jesus, your love brings freedom and pard...",0


Distribution of labels

In [4]:
tweet_df["target"].value_counts()

0    9256
1    2114
Name: target, dtype: int64

## Generating Train and Validation sets

In [5]:
x_train, x_val, y_train, y_val = train_test_split(tweet_df["text"].values, tweet_df["target"].values, test_size=0.2)

## Tokenizing the tweets

In [ ]:
tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')

In [7]:
''' truncation=True and padding=True will ensure that all sequences are padded to the same length and 
 are truncated to be no longer than the model’s maximum input length '''

train_encodings = tokenizer(list(x_train), truncation=True, padding=True)
val_encodings = tokenizer(list(x_val), truncation=True, padding=True)

## Creating a Dataset object using the encodings and labels

In [8]:
train_dataset = tf.data.Dataset.from_tensor_slices((
    dict(train_encodings),
    y_train
))

val_dataset = tf.data.Dataset.from_tensor_slices((
    dict(val_encodings),
    y_val
))

# Fine-tuning the pretrained model using native TensorFlow
The classifier is built by fine-tuning a pre-trained DistilBert model.

In [ ]:
model = TFDistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=2)
optimizer = tf.keras.optimizers.Adam(learning_rate=5e-5)

model.compile(optimizer=optimizer, loss=model.compute_loss, metrics=["accuracy"])

In [16]:
model.fit(train_dataset.shuffle(1000).batch(16), epochs=5, validation_data=val_dataset.shuffle(1000).batch(16))

The parameters `output_attentions`, `output_hidden_states` and `use_cache` cannot be updated when calling a model.They have to be set to True/False in the config object (i.e.: `config=XConfig.from_pretrained('name', output_attentions=True)`).
The parameter `return_dict` cannot be set in graph mode and will always be set to `True`.


Epoch 1/5


The parameters `output_attentions`, `output_hidden_states` and `use_cache` cannot be updated when calling a model.They have to be set to True/False in the config object (i.e.: `config=XConfig.from_pretrained('name', output_attentions=True)`).
The parameter `return_dict` cannot be set in graph mode and will always be set to `True`.


569/569 [==============================] - ETA: 0s - loss: 0.3272 - accuracy: 0.8611

The parameters `output_attentions`, `output_hidden_states` and `use_cache` cannot be updated when calling a model.They have to be set to True/False in the config object (i.e.: `config=XConfig.from_pretrained('name', output_attentions=True)`).
The parameter `return_dict` cannot be set in graph mode and will always be set to `True`.


569/569 [==============================] - 92s 149ms/step - loss: 0.3271 - accuracy: 0.8612 - val_loss: 0.2035 - val_accuracy: 0.9160
Epoch 2/5
569/569 [==============================] - 85s 150ms/step - loss: 0.1588 - accuracy: 0.9371 - val_loss: 0.2246 - val_accuracy: 0.9204
Epoch 3/5
569/569 [==============================] - 86s 152ms/step - loss: 0.0768 - accuracy: 0.9736 - val_loss: 0.2381 - val_accuracy: 0.9085
Epoch 4/5
569/569 [==============================] - 87s 153ms/step - loss: 0.0411 - accuracy: 0.9873 - val_loss: 0.2800 - val_accuracy: 0.9200
Epoch 5/5
569/569 [==============================] - 87s 153ms/step - loss: 0.0240 - accuracy: 0.9928 - val_loss: 0.2981 - val_accuracy: 0.9156
